# Import Libraries

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.impute import KNNImputer

# Obtain Data

Read csv with election results into pandas, scrub county names to match formatting of census data, filter for 2020 Trump vote proportion and drop unneeded columns

In [2]:
election_csv_df = pd.read_csv('Data/election_results.csv')
election_csv_df = election_csv_df[(election_csv_df['year']==2020) & (election_csv_df['candidate']=='DONALD J TRUMP')]
election_csv_df['County'] = election_csv_df.apply(lambda x: x['county_name'].title() + ' County, ' + x['state'].title(),
                                            axis=1)
election_csv_df['County'] = election_csv_df['County'].apply(lambda x: x.replace('City County', 'City'))
election_csv_df['County'] = election_csv_df['County'].apply(lambda x: x.replace('County County', 'County'))
election_csv_df['County'] = election_csv_df['County'].apply(lambda x: x.replace('Saint', 'St.'))
election_csv_df['County'] = election_csv_df['County'].apply(lambda x: x.replace('St ', 'St. '))
election_csv_df['County'] = election_csv_df['County'].apply(lambda x: x.replace('District Of Columbia County', 'District Of Columbia'))
election_csv_df = election_csv_df[['County', 'candidatevotes', 'totalvotes', 'mode']]

election_df = pd.DataFrame(election_csv_df.groupby(by='County', axis=0).sum()['candidatevotes'])
election_df.reset_index(inplace=True)
election_df.columns=['County', 'Trump Votes']
election_df['Total Votes'] = election_csv_df.groupby(by='County', axis=0).max()['totalvotes'].values

print('election_df shape:', election_df.shape)
election_df.head()

election_df shape: (3155, 3)


,County,Trump Votes,Total Votes
0,"Abbeville County, South Carolina",8215.0,12433.0
1,"Acadia County, Louisiana",22596.0,28425.0
2,"Accomack County, Virginia",9172.0,16962.0
3,"Ada County, Idaho",130699.0,259389.0
4,"Adair County, Iowa",2922.0,4177.0


Read csv with county areas into pandas, scrub county names to match formatting of census data and drop unneeded columns

In [3]:
area_df = pd.read_csv('Data/area.csv')
area_df = area_df[['Areaname', 'LND010200D']]
area_df.columns = ['County', 'Area']
area_df['County']

state_abbrev_df = pd.read_csv('Data/state_abbrev.csv')
def replace_abbrev(row):
    if len(row.split(',')) == 1:
        return row    
    else:
        state_abbrev = row.split(',')[1][1:]
        state = state_abbrev_df[state_abbrev_df['Code']==state_abbrev]['State'].values[0]
        return row.split(',')[0] + ' County, ' + state
area_df['County'] = area_df['County'].apply(replace_abbrev)
area_df['County'] = area_df['County'].apply(lambda x: x.title())
area_df['County'] = area_df['County'].apply(lambda x: x.replace('District Of Columbia', 'District Of Columbia, District Of Columbia'))
area_df['County'] = area_df['County'].apply(lambda x: x.replace('City County', 'City'))
print('area_df shape:', area_df.shape)
area_df.head()

area_df shape: (3198, 2)


,County,Area
0,United States,3794083.06
1,Alabama,52419.02
2,"Autauga County, Alabama",604.45
3,"Baldwin County, Alabama",2026.93
4,"Barbour County, Alabama",904.52


Update Virginia independent cities in area_df table with city instead of County to match formatting of other tables

In [4]:
area_df['Duplicated'] = area_df['County'].duplicated()
duplicate_counties = area_df[area_df['Duplicated']==1]['County'].values

def virginia_cities(row):
    if row['County'] not in duplicate_counties:
        return row
    elif row['Area'] == max(area_df[area_df['County']==row['County']]['Area'].values):
        return row
    else:
        row['County'] = row['County'].replace('County', 'City')
        return row
area_df = area_df.apply(virginia_cities, axis=1)
area_df.drop('Duplicated', inplace=True, axis=1)
area_df.drop_duplicates(subset='County', inplace=True)
area_df.head()

,County,Area
0,United States,3794083.06
1,Alabama,52419.02
2,"Autauga County, Alabama",604.45
3,"Baldwin County, Alabama",2026.93
4,"Barbour County, Alabama",904.52


Read csv with county level education data into pandas and drop unneeded columns

In [5]:
education_df = pd.read_csv('Data/education.csv', header=1)
education_df = education_df[['Geographic Area Name',
                            'Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over', 
                            "Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!Bachelor's degree or higher"]]
education_df.columns = ['County', 'Persons 25+', 'Persons 25+ w/ Bachelors Degree']
print('education_df shape: ', education_df.shape)
education_df.head()

education_df shape:  (840, 3)


,County,Persons 25+,Persons 25+ w/ Bachelors Degree
0,"Baldwin County, Alabama",159717,51471
1,"Calhoun County, Alabama",79084,15257
2,"Cullman County, Alabama",58795,9241
3,"DeKalb County, Alabama",47007,5999
4,"Elmore County, Alabama",57553,14310


Read csv with county level economic data into pandas and drop unneeded columns

In [6]:
economic_df = pd.read_csv('Data/economics.csv', header=1)
economic_df = economic_df[['Geographic Area Name',
                          'Estimate!!INCOME AND BENEFITS (IN 2019 INFLATION-ADJUSTED DOLLARS)!!Per capita income (dollars)',
                          'Estimate!!EMPLOYMENT STATUS!!Population 16 years and over!!In labor force',
                          'Estimate!!EMPLOYMENT STATUS!!Population 16 years and over!!In labor force!!Civilian labor force!!Unemployed',]]
economic_df.columns = ['County', 'Per Capita Income', 'Persons 16+ in Labor Force', 'Persons 16+ Unemployed',]
print('economic_df shape: ', economic_df.shape)
economic_df.head()

economic_df shape:  (840, 4)


,County,Per Capita Income,Persons 16+ in Labor Force,Persons 16+ Unemployed
0,"Baldwin County, Alabama",32443,101561,5549
1,"Calhoun County, Alabama",24579,51001,3754
2,"Cullman County, Alabama",23968,38222,1783
3,"DeKalb County, Alabama",21939,34492,1606
4,"Elmore County, Alabama",31396,37693,1014


Read csv with ethnicity data into pandas and drop unneeded columns

In [7]:
ethnicity_df = pd.read_csv('Data/ethnicity.csv', header=1)
ethnicity_df = ethnicity_df[['Geographic Area Name',
                             'Estimate!!SEX AND AGE!!Total population!!Sex ratio (males per 100 females)',
                             'Estimate!!SEX AND AGE!!Total population',
                             'Estimate!!HISPANIC OR LATINO AND RACE!!Total population!!Hispanic or Latino (of any race)',
                             'Estimate!!RACE!!Total population!!One race!!White']]
ethnicity_df.columns = ['County', 'Sex Ratio (M/F)', 'Total Persons', 'Hispanic Persons', 'White Persons']
print('ethnicity_df shape: ', ethnicity_df.shape)
ethnicity_df.head()

ethnicity_df shape:  (840, 5)


,County,Sex Ratio (M/F),Total Persons,Hispanic Persons,White Persons
0,"Baldwin County, Alabama",95.7,223234,10534,190912
1,"Calhoun County, Alabama",91.5,113605,4614,82323
2,"Cullman County, Alabama",94.0,83768,3752,N
3,"DeKalb County, Alabama",99.6,71513,10775,59305
4,"Elmore County, Alabama",97.4,81209,2563,61634


Read csv with disability data into pandas and drop unneeded columns

In [8]:
disability_df = pd.read_csv('Data/disability.csv', header=1)
disability_df = disability_df[['Geographic Area Name',
    'Estimate!!Percent with a disability!!Total civilian noninstitutionalized population']]
disability_df.columns = ['County', 'Disability Proportion']
print('disability_df shape: ', disability_df.shape)
disability_df.head()

disability_df shape:  (840, 2)


,County,Disability Proportion
0,"Baldwin County, Alabama",14.4
1,"Calhoun County, Alabama",20.0
2,"Cullman County, Alabama",17.5
3,"DeKalb County, Alabama",10.8
4,"Elmore County, Alabama",12.9


Read csv with demographic data into pandas and drop unneeded columns

In [9]:
demographics_df = pd.read_csv('Data/demographics.csv', header=1)
demographics_df = demographics_df[['Geographic Area Name',
                                  'Estimate!!Total!!Total population!!SUMMARY INDICATORS!!Median age (years)']]
demographics_df.columns = ['County', 'Median Age']
print('demographics_df shape: ', demographics_df.shape)
demographics_df.head()

demographics_df shape:  (840, 2)


,County,Median Age
0,"Baldwin County, Alabama",43.0
1,"Calhoun County, Alabama",39.6
2,"Cullman County, Alabama",41.9
3,"DeKalb County, Alabama",37.7
4,"Elmore County, Alabama",39.0


Read csv with veteran proprtion into pandas and drop unneeded columns

In [10]:
veterans_df = pd.read_csv('Data/veterans.csv', header=1)
veterans_df = veterans_df[['Geographic Area Name',
                           'Estimate!!Total!!Civilian population 18 years and over',
                           'Estimate!!Veterans!!Civilian population 18 years and over']]
veterans_df.columns = ['County', 'Civilians 18+', 'Veterans 18+']
print('veterans_df shape: ', veterans_df.shape)
veterans_df.head()

veterans_df shape:  (840, 3)


,County,Civilians 18+,Veterans 18+
0,"Baldwin County, Alabama",176331,19580
1,"Calhoun County, Alabama",87525,8552
2,"Cullman County, Alabama",64955,4907
3,"DeKalb County, Alabama",53737,3499
4,"Elmore County, Alabama",61837,6512


Read csv with employment data into pandas and drop unneeded columns

In [11]:
occupation_df = pd.read_csv('Data/occupation.csv', header=1)
occupation_df = occupation_df[['Geographic Area Name',
                           'Estimate!!Total!!Civilian employed population 16 years and over',
                           'Estimate!!Total!!Civilian employed population 16 years and over!!Agriculture, forestry, fishing and hunting, and mining:!!Agriculture, forestry, fishing and hunting',
                           'Estimate!!Total!!Civilian employed population 16 years and over!!Agriculture, forestry, fishing and hunting, and mining:!!Mining, quarrying, and oil and gas extraction',
                           'Estimate!!Total!!Civilian employed population 16 years and over!!Manufacturing']]
occupation_df.columns = ['County', 'Employees 16+', 'Agriculture Employees 16+', 'Mining Employees 16+', 
                          'Manufacturing Employees 16+']
print('occupation_df shape: ', occupation_df.shape)
occupation_df.head()

occupation_df shape:  (840, 5)


,County,Employees 16+,Agriculture Employees 16+,Mining Employees 16+,Manufacturing Employees 16+
0,"Baldwin County, Alabama",96012,1090,205,8791
1,"Calhoun County, Alabama",45641,298,714,7044
2,"Cullman County, Alabama",36439,399,0,5282
3,"DeKalb County, Alabama",32544,845,7,9052
4,"Elmore County, Alabama",35358,101,77,4634


Merge all dfs from census sources together

In [12]:
df = pd.merge(education_df, economic_df, on='County')
df = pd.merge(df, ethnicity_df, on='County')
df = pd.merge(df, disability_df, on='County')
df = pd.merge(df, demographics_df, on='County')
df = pd.merge(df, veterans_df, on='County')
df = pd.merge(df, occupation_df, on='County')
print('df shape:', df.shape)
df.head()

df shape: (840, 18)


,County,Persons 25+,Persons 25+ w/ Bachelors Degree,Per Capita Income,Persons 16+ in Labor Force,Persons 16+ Unemployed,Sex Ratio (M/F),Total Persons,Hispanic Persons,White Persons,Disability Proportion,Median Age,Civilians 18+,Veterans 18+,Employees 16+,Agriculture Employees 16+,Mining Employees 16+,Manufacturing Employees 16+
0,"Baldwin County, Alabama",159717,51471,32443,101561,5549,95.7,223234,10534,190912,14.4,43.0,176331,19580,96012,1090,205,8791
1,"Calhoun County, Alabama",79084,15257,24579,51001,3754,91.5,113605,4614,82323,20.0,39.6,87525,8552,45641,298,714,7044
2,"Cullman County, Alabama",58795,9241,23968,38222,1783,94.0,83768,3752,N,17.5,41.9,64955,4907,36439,399,0,5282
3,"DeKalb County, Alabama",47007,5999,21939,34492,1606,99.6,71513,10775,59305,10.8,37.7,53737,3499,32544,845,7,9052
4,"Elmore County, Alabama",57553,14310,31396,37693,1014,97.4,81209,2563,61634,12.9,39.0,61837,6512,35358,101,77,4634


Scrub county names in df with census data to match county names in other dfs for subsequent merger

In [13]:
def scrub_county(name):
    name = name.replace('Parish', 'County')
    name = name.replace('ñ', 'n')
    name = name.replace('Saint', 'St.')
    name = name.replace('City County', 'City')
    name = name.title()
    return name
df['County'] = df['County'].apply(scrub_county)

Merge df with election results

In [14]:
df = pd.merge(df, election_df, on='County')
print('df shape:', df.shape)
df.head()

df shape: (826, 20)


,County,Persons 25+,Persons 25+ w/ Bachelors Degree,Per Capita Income,Persons 16+ in Labor Force,Persons 16+ Unemployed,Sex Ratio (M/F),Total Persons,Hispanic Persons,White Persons,Disability Proportion,Median Age,Civilians 18+,Veterans 18+,Employees 16+,Agriculture Employees 16+,Mining Employees 16+,Manufacturing Employees 16+,Trump Votes,Total Votes
0,"Baldwin County, Alabama",159717,51471,32443,101561,5549,95.7,223234,10534,190912,14.4,43.0,176331,19580,96012,1090,205,8791,83544.0,109679.0
1,"Calhoun County, Alabama",79084,15257,24579,51001,3754,91.5,113605,4614,82323,20.0,39.6,87525,8552,45641,298,714,7044,35101.0,50983.0
2,"Cullman County, Alabama",58795,9241,23968,38222,1783,94.0,83768,3752,N,17.5,41.9,64955,4907,36439,399,0,5282,36880.0,41851.0
3,"Dekalb County, Alabama",47007,5999,21939,34492,1606,99.6,71513,10775,59305,10.8,37.7,53737,3499,32544,845,7,9052,24767.0,29356.0
4,"Elmore County, Alabama",57553,14310,31396,37693,1014,97.4,81209,2563,61634,12.9,39.0,61837,6512,35358,101,77,4634,30164.0,41030.0


Scrub area_df county names to match df county names

In [15]:
missing_counties = pd.merge(area_df, df, on='County', how='right')[pd.merge(area_df, df, on='County', how='right').isna().max(axis=1)]['County'].values
replacement_counties = []
for county in missing_counties:
    replacement_counties.append(county.replace('City', 'County'))
for num in range(0, len(missing_counties)):
    area_df['County'] = area_df['County'].apply(lambda x: x.replace(replacement_counties[num], missing_counties[num]))

Merge df with area_df

In [16]:
df = pd.merge(df, area_df, on='County')
print('df shape:', df.shape)
df.head()

df shape: (826, 21)


,County,Persons 25+,Persons 25+ w/ Bachelors Degree,Per Capita Income,Persons 16+ in Labor Force,Persons 16+ Unemployed,Sex Ratio (M/F),Total Persons,Hispanic Persons,White Persons,...,Median Age,Civilians 18+,Veterans 18+,Employees 16+,Agriculture Employees 16+,Mining Employees 16+,Manufacturing Employees 16+,Trump Votes,Total Votes,Area
0,"Baldwin County, Alabama",159717,51471,32443,101561,5549,95.7,223234,10534,190912,...,43.0,176331,19580,96012,1090,205,8791,83544.0,109679.0,2026.93
1,"Calhoun County, Alabama",79084,15257,24579,51001,3754,91.5,113605,4614,82323,...,39.6,87525,8552,45641,298,714,7044,35101.0,50983.0,612.32
2,"Cullman County, Alabama",58795,9241,23968,38222,1783,94.0,83768,3752,N,...,41.9,64955,4907,36439,399,0,5282,36880.0,41851.0,754.82
3,"Dekalb County, Alabama",47007,5999,21939,34492,1606,99.6,71513,10775,59305,...,37.7,53737,3499,32544,845,7,9052,24767.0,29356.0,778.65
4,"Elmore County, Alabama",57553,14310,31396,37693,1014,97.4,81209,2563,61634,...,39.0,61837,6512,35358,101,77,4634,30164.0,41030.0,657.21


# Scrub Data

Use df.info() to see what data scrubbing needed

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 826 entries, 0 to 825
Data columns (total 21 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   County                           826 non-null    object 
 1   Persons 25+                      826 non-null    int64  
 2   Persons 25+ w/ Bachelors Degree  826 non-null    int64  
 3   Per Capita Income                826 non-null    int64  
 4   Persons 16+ in Labor Force       826 non-null    int64  
 5   Persons 16+ Unemployed           826 non-null    int64  
 6   Sex Ratio (M/F)                  826 non-null    float64
 7   Total Persons                    826 non-null    int64  
 8   Hispanic Persons                 826 non-null    int64  
 9   White Persons                    769 non-null    float64
 10  Disability Proportion            826 non-null    float64
 11  Median Age                       826 non-null    float64
 12  Civilians 18+         

Replace all N datapoints, signifying a missing value, with np.nan and change column data types to int

In [18]:
replace_dict = {'N': np.nan}
df = df.replace(replace_dict)
dtype_dict = {'White Persons': float, 'Employees 16+': float, 'Agriculture Employees 16+': float,
              'Mining Employees 16+': float, 'Manufacturing Employees 16+': float}
df = df.astype(dtype_dict)
df.head()

,County,Persons 25+,Persons 25+ w/ Bachelors Degree,Per Capita Income,Persons 16+ in Labor Force,Persons 16+ Unemployed,Sex Ratio (M/F),Total Persons,Hispanic Persons,White Persons,...,Median Age,Civilians 18+,Veterans 18+,Employees 16+,Agriculture Employees 16+,Mining Employees 16+,Manufacturing Employees 16+,Trump Votes,Total Votes,Area
0,"Baldwin County, Alabama",159717,51471,32443,101561,5549,95.7,223234,10534,190912.0,...,43.0,176331,19580,96012.0,1090.0,205.0,8791.0,83544.0,109679.0,2026.93
1,"Calhoun County, Alabama",79084,15257,24579,51001,3754,91.5,113605,4614,82323.0,...,39.6,87525,8552,45641.0,298.0,714.0,7044.0,35101.0,50983.0,612.32
2,"Cullman County, Alabama",58795,9241,23968,38222,1783,94.0,83768,3752,NaN,...,41.9,64955,4907,36439.0,399.0,0.0,5282.0,36880.0,41851.0,754.82
3,"Dekalb County, Alabama",47007,5999,21939,34492,1606,99.6,71513,10775,59305.0,...,37.7,53737,3499,32544.0,845.0,7.0,9052.0,24767.0,29356.0,778.65
4,"Elmore County, Alabama",57553,14310,31396,37693,1014,97.4,81209,2563,61634.0,...,39.0,61837,6512,35358.0,101.0,77.0,4634.0,30164.0,41030.0,657.21


Fill in missing vote total for San Joaquin County, California with manually researched number

In [19]:
df.loc[df[df['County']=='San Joaquin County, California'].index, 'Total Votes'] = 288492
df[df['County']=='San Joaquin County, California']

,County,Persons 25+,Persons 25+ w/ Bachelors Degree,Per Capita Income,Persons 16+ in Labor Force,Persons 16+ Unemployed,Sex Ratio (M/F),Total Persons,Hispanic Persons,White Persons,...,Median Age,Civilians 18+,Veterans 18+,Employees 16+,Agriculture Employees 16+,Mining Employees 16+,Manufacturing Employees 16+,Trump Votes,Total Votes,Area
67,"San Joaquin County, California",484712,96795,28919,348301,21705,99.9,762148,320334,431364.0,...,34.7,558269,25650,326476.0,16456.0,447.0,26476.0,121098.0,288492.0,1426.25


Fill in miising area for Broomfield County, Colorado with manually researched number

In [20]:
df.loc[df[df['County']=='Broomfield County, Colorado'].index, 'Area'] = 33.03
df[df['County']=='Broomfield County, Colorado']

,County,Persons 25+,Persons 25+ w/ Bachelors Degree,Per Capita Income,Persons 16+ in Labor Force,Persons 16+ Unemployed,Sex Ratio (M/F),Total Persons,Hispanic Persons,White Persons,...,Median Age,Civilians 18+,Veterans 18+,Employees 16+,Agriculture Employees 16+,Mining Employees 16+,Manufacturing Employees 16+,Trump Votes,Total Votes,Area
86,"Broomfield County, Colorado",49543,28176,57213,42235,832,102.0,70465,8964,60615.0,...,38.9,54923,3876,41403.0,146.0,109.0,3734.0,16295.0,46632.0,33.03


Replace all 0 Agriculture Employees 16+ and Mining Employees 16+ values with np.nan to subsequently impute

In [21]:
zero_replace_dict = {0: np.nan}
df['Agriculture Employees 16+'] = df['Agriculture Employees 16+'].replace(zero_replace_dict)
df['Mining Employees 16+'] = df['Mining Employees 16+'].replace(zero_replace_dict)
df.head()

,County,Persons 25+,Persons 25+ w/ Bachelors Degree,Per Capita Income,Persons 16+ in Labor Force,Persons 16+ Unemployed,Sex Ratio (M/F),Total Persons,Hispanic Persons,White Persons,...,Median Age,Civilians 18+,Veterans 18+,Employees 16+,Agriculture Employees 16+,Mining Employees 16+,Manufacturing Employees 16+,Trump Votes,Total Votes,Area
0,"Baldwin County, Alabama",159717,51471,32443,101561,5549,95.7,223234,10534,190912.0,...,43.0,176331,19580,96012.0,1090.0,205.0,8791.0,83544.0,109679.0,2026.93
1,"Calhoun County, Alabama",79084,15257,24579,51001,3754,91.5,113605,4614,82323.0,...,39.6,87525,8552,45641.0,298.0,714.0,7044.0,35101.0,50983.0,612.32
2,"Cullman County, Alabama",58795,9241,23968,38222,1783,94.0,83768,3752,NaN,...,41.9,64955,4907,36439.0,399.0,NaN,5282.0,36880.0,41851.0,754.82
3,"Dekalb County, Alabama",47007,5999,21939,34492,1606,99.6,71513,10775,59305.0,...,37.7,53737,3499,32544.0,845.0,7.0,9052.0,24767.0,29356.0,778.65
4,"Elmore County, Alabama",57553,14310,31396,37693,1014,97.4,81209,2563,61634.0,...,39.0,61837,6512,35358.0,101.0,77.0,4634.0,30164.0,41030.0,657.21


Calculate interaction independent variables and drop variables used to calculate interacitons

In [24]:
df['Bachelors Degree Proportion'] = df['Persons 25+ w/ Bachelors Degree']/df['Persons 25+']
df.drop(columns=['Persons 25+ w/ Bachelors Degree', 'Persons 25+'], inplace=True)

df['Unemployment Rate'] = df['Persons 16+ Unemployed']/df['Persons 16+ in Labor Force']
df.drop(columns=['Persons 16+ Unemployed', 'Persons 16+ in Labor Force'], inplace=True)

df['Hispanic Population Proportion'] = df['Hispanic Persons']/df['Total Persons']
df.drop(columns=['Hispanic Persons'], inplace=True)

df['White Population Proportion'] = df['White Persons']/df['Total Persons']
df.drop(columns=['White Persons'], inplace=True)

df['Veteran Population Proportion'] = df['Veterans 18+']/df['Civilians 18+']
df.drop(columns=['Veterans 18+', 'Civilians 18+'], inplace=True)

df['Agriculture Employment Proportion'] = df['Agriculture Employees 16+']/df['Employees 16+']
df.drop(columns=['Agriculture Employees 16+'], inplace=True)

df['Mining Employment Proportion'] = df['Mining Employees 16+']/df['Employees 16+']
df.drop(columns=['Mining Employees 16+'], inplace=True)

df['Manufacturing Employment Proportion'] = df['Manufacturing Employees 16+']/df['Employees 16+']
df.drop(columns=['Manufacturing Employees 16+', 'Employees 16+'], inplace=True)

df['Trump Vote Proportion'] = df['Trump Votes']/df['Total Votes']
df.drop(columns=['Trump Votes', 'Total Votes'], inplace=True)

df['Population Density'] = df['Total Persons']/df['Area']
df.drop(columns=['Total Persons', 'Area'], inplace=True)

df.head()

,County,Per Capita Income,Sex Ratio (M/F),Disability Proportion,Median Age,Bachelors Degree Proportion,Unemployment Rate,Hispanic Population Proportion,White Population Proportion,Veteran Population Proportion,Agriculture Employment Proportion,Mining Employment Proportion,Manufacturing Employment Proportion,Trump Vote Proportion,Population Density
0,"Baldwin County, Alabama",32443,95.7,14.4,43.0,0.322264,0.054637,0.047188,0.855210,0.111041,0.011353,0.002135,0.091561,0.761714,110.134045
1,"Calhoun County, Alabama",24579,91.5,20.0,39.6,0.192921,0.073606,0.040614,0.724642,0.097709,0.006529,0.015644,0.154335,0.688484,185.532075
2,"Cullman County, Alabama",23968,94.0,17.5,41.9,0.157173,0.046649,0.044790,NaN,0.075545,0.010950,NaN,0.144955,0.881221,110.977452
3,"Dekalb County, Alabama",21939,99.6,10.8,37.7,0.127619,0.046562,0.150672,0.829290,0.065113,0.025965,0.000215,0.278147,0.843678,91.842291
4,"Elmore County, Alabama",31396,97.4,12.9,39.0,0.248640,0.026902,0.031561,0.758955,0.105309,0.002856,0.002178,0.131059,0.735169,123.566288


Drop columns not useful for modeling

In [25]:
df.drop(columns=['County'], inplace=True)

Impute missing values

In [26]:
imputer_ss = StandardScaler()
df = pd.DataFrame(imputer_ss.fit_transform(df), columns=df.columns)

imputer = KNNImputer()
df = pd.DataFrame(imputer.fit_transform(df), columns=df.columns)

df.head()

,Per Capita Income,Sex Ratio (M/F),Disability Proportion,Median Age,Bachelors Degree Proportion,Unemployment Rate,Hispanic Population Proportion,White Population Proportion,Veteran Population Proportion,Agriculture Employment Proportion,Mining Employment Proportion,Manufacturing Employment Proportion,Trump Vote Proportion,Population Density
0,-0.148144,-0.284776,0.227506,0.796274,0.134635,0.602900,-0.555534,0.523370,1.104469,-0.094356,-0.278666,-0.317551,1.545359,-0.243821
1,-1.083999,-1.130981,1.854012,0.070523,-1.047594,1.736941,-0.603534,-0.346205,0.636337,-0.315298,0.505081,0.766322,1.059934,-0.211849
2,-1.156711,-0.627287,1.127893,0.561472,-1.374344,0.125319,-0.573042,0.481227,-0.141941,-0.112813,0.078858,0.604357,2.337556,-0.243463
3,-1.398172,0.500986,-0.818106,-0.335043,-1.644476,0.120117,0.200073,0.350741,-0.508217,0.574954,-0.390064,2.904105,2.088684,-0.251578
4,-0.272742,0.057736,-0.208166,-0.057550,-0.538306,-1.055215,-0.669643,-0.117683,0.903197,-0.483528,-0.276196,0.364437,1.369401,-0.238125


Confirm data is scrubbed with df.info()

In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 826 entries, 0 to 825
Data columns (total 14 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   Per Capita Income                    826 non-null    float64
 1   Sex Ratio (M/F)                      826 non-null    float64
 2   Disability Proportion                826 non-null    float64
 3   Median Age                           826 non-null    float64
 4   Bachelors Degree Proportion          826 non-null    float64
 5   Unemployment Rate                    826 non-null    float64
 6   Hispanic Population Proportion       826 non-null    float64
 7   White Population Proportion          826 non-null    float64
 8   Veteran Population Proportion        826 non-null    float64
 9   Agriculture Employment Proportion    826 non-null    float64
 10  Mining Employment Proportion         826 non-null    float64
 11  Manufacturing Employment Proport

# Explore Data

In [23]:
df.describe()

,Persons 25+,Persons 25+ w/ Bachelors Degree,Per Capita Income,Persons 16+ in Labor Force,Persons 16+ Unemployed,Sex Ratio (M/F),Total Persons,Hispanic Persons,White Persons,Disability Proportion,Median Age,Civilians 18+,Veterans 18+,Employees 16+,Agriculture Employees 16+,Mining Employees 16+,Manufacturing Employees 16+,Trump Votes,Total Votes,Area
count,8.260000e+02,8.260000e+02,826.000000,8.260000e+02,826.000000,826.000000,8.260000e+02,8.260000e+02,7.690000e+02,826.000000,826.000000,8.260000e+02,826.000000,8.220000e+02,807.000000,616.000000,822.000000,8.260000e+02,8.260000e+02,826.000000
mean,2.309119e+05,8.170093e+04,33687.851090,1.752166e+05,7863.941889,97.113438,3.378010e+05,6.824185e+04,2.484945e+05,13.616707,39.269613,2.611782e+05,17209.616223,1.666972e+05,1404.452292,792.305195,15463.096107,7.060205e+04,1.627407e+05,1123.648196
std,4.057779e+05,1.538831e+05,8408.098692,3.124600e+05,15420.494452,4.966343,5.904087e+05,2.431972e+05,3.635882e+05,3.445047,4.687643,4.578001e+05,22332.441500,2.968614e+05,3176.977135,3411.434041,26242.781712,8.979623e+04,2.605114e+05,1634.951036
min,3.729700e+04,4.923000e+03,15589.000000,2.028400e+04,388.000000,84.200000,6.238500e+04,2.470000e+02,8.354000e+03,5.800000,25.100000,4.330600e+04,1801.000000,1.876700e+04,1.000000,7.000000,202.000000,7.801000e+03,2.091800e+04,15.410000
25%,6.535275e+04,1.632625e+04,28126.000000,4.661700e+04,1986.750000,94.125000,9.648375e+04,4.900500e+03,8.380100e+04,11.200000,36.600000,7.419525e+04,6015.250000,4.403375e+04,373.000000,67.000000,4501.500000,2.680975e+04,4.691425e+04,474.335000
50%,1.098415e+05,3.229050e+04,32221.000000,8.115800e+04,3435.500000,96.700000,1.603765e+05,1.252500e+04,1.406970e+05,13.300000,39.000000,1.246190e+05,9957.500000,7.692450e+04,699.000000,180.000000,8496.000000,4.220400e+04,8.223550e+04,691.250000
75%,2.310868e+05,7.862925e+04,37359.500000,1.787118e+05,7676.750000,99.200000,3.463982e+05,4.266025e+04,2.723550e+05,15.700000,41.675000,2.612122e+05,19919.750000,1.698220e+05,1273.000000,497.000000,16455.000000,7.977350e+04,1.719050e+05,1043.065000
max,6.961614e+06,2.349549e+06,82720.000000,5.313215e+06,267870.000000,144.300000,1.003911e+07,4.881970e+06,5.234081e+06,27.600000,68.900000,7.887028e+06,251329.000000,5.037815e+06,40919.000000,73803.000000,433018.000000,1.145530e+06,4.264365e+06,20105.320000


In [24]:
df[df['Area']==0]

,County,Persons 25+,Persons 25+ w/ Bachelors Degree,Per Capita Income,Persons 16+ in Labor Force,Persons 16+ Unemployed,Sex Ratio (M/F),Total Persons,Hispanic Persons,White Persons,...,Median Age,Civilians 18+,Veterans 18+,Employees 16+,Agriculture Employees 16+,Mining Employees 16+,Manufacturing Employees 16+,Trump Votes,Total Votes,Area
86,"Broomfield County, Colorado",49543,28176,57213,42235,832,102.0,70465,8964,60615.0,...,38.9,54923,3876,41403.0,146.0,109.0,3734.0,16295.0,46632.0,0.0


In [37]:
jbb = pd.read_csv('Data/election_results.csv')
jbb = jbb[(jbb['year']==2020) & (jbb['candidate']=='DONALD J TRUMP')]
jbb.reset_index(inplace=True)
print(jbb.shape)
jbb.head(50)
jbb.to_csv('jbb.csv')

(5117, 13)


In [38]:
jbb['totalvotes'].sort_values()

3534         66.0
3518        159.0
3514        194.0
3899        284.0
3955        302.0
          ...    
130     2068144.0
129     2068144.0
1353    2321486.0
471     4264365.0
491           NaN
Name: totalvotes, Length: 5117, dtype: float64

In [49]:
jbb.iloc[502]

index                      52904
year                        2020
state                 CALIFORNIA
state_po                      CA
county_name           STANISLAUS
county_fips               6099.0
office                 PRESIDENT
candidate         DONALD J TRUMP
party                 REPUBLICAN
candidatevotes          104145.0
totalvotes              214954.0
version                 20210622
mode                       TOTAL
Name: 502, dtype: object